In [1]:
import pandas as pd
from tqdm import tqdm
from database.strategy import Strategy
from database.market import Market
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta, timezone
import math
import numpy as np

In [2]:
strat_db = Strategy("unity")
market = Market()

In [3]:
market.connect()
sp5 = market.retrieve_data("sp500")
market.close()

In [37]:
start = datetime(2019,1,1)
end = datetime(2020,1,1)
reload = True
datasets = [
            "pdr",
#             "tiingo",
#             "finnhub"
]
yearly_delta_range = range(50,155,50)
ceiling_settings = [True,False]
bond_settings = [True,False]
bond_value_settings = [True,False]
bond_classification_settings = [True,False]
weekly_settings = [True,False]
weekly_value_settings = [True,False]
weekly_classification_settings = [True,False]
fundamental_settings = [True,False]
fundamental_value_settings = [True,False]
fundamental_classification_settings = [True,False]
exit_settings = [True,False]
dataset = "pdr"
initial = 100

In [13]:
strat_db.connect()
bonds = strat_db.retrieve_data("{}_bond_weekly_sim".format(dataset))
fundamentals = strat_db.retrieve_data("fundamental_quarterly_sim".format(dataset))
factors = strat_db.retrieve_data("{}_weekly_sim".format("pdr"))
strat_db.close()

In [28]:
bonds

,_id,year,quarter,week,weekly_bond_classification_prediction,weekly_bond_classification_score,weekly_bond_regression_prediction,weekly_bond_regression_score,ticker
0,6066b73c6fa4fdd0ace111c5,2021,2,13,0,0.733333,748.879944,0.054234,A
1,6066b73c6fa4fdd0ace111c6,2021,2,13,1,0.600000,201.620789,0.103985,AAL
2,6066b73c6fa4fdd0ace111c7,2021,2,13,1,0.800000,1115.105591,0.033725,AAP
3,6066b73c6fa4fdd0ace111c8,2021,2,13,1,0.666667,647.523376,0.064818,AAPL
4,6066b73c6fa4fdd0ace111c9,2021,2,13,1,0.733333,109.981552,0.023198,ABBV
...,...,...,...,...,...,...,...,...,...
12467,6070598bf7065868edb8305b,2020,2,22,1,0.604167,1132.565796,0.207096,BKNG
12468,6070598bf7065868edb8305c,2020,2,23,1,0.604167,1132.565796,0.207096,BKNG
12469,6070598bf7065868edb8305d,2020,2,24,1,0.604167,1132.565796,0.207096,BKNG
12470,6070598bf7065868edb8305e,2020,2,25,1,0.604167,1132.565796,0.207096,BKNG


In [17]:
for dataset in tqdm(datasets):
    if reload:
        market.connect()
        prices = market.retrieve_data("{}_prices".format(dataset))
        market.close()
        if dataset == "pdr":
            prices = ColumnTransformer.rename_columns(prices," ")
        else:
            prices = ColumnTransformer.rename_columns(prices,"")
        prices = DateTransformer.convert_to_date(dataset,prices,"date")
        prices["year"] = [x.year for x in prices["date"]]
        prices["week"] = [x.week for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        sim = prices.drop("_id",axis=1).merge(factors.drop("_id",axis=1),on=["year","quarter","week","ticker"],how="left").dropna() \
        .merge(bonds,on=["year","quarter","week","ticker"],how="left").dropna()
        sim["weekly_predicted_delta"] = (sim["weekly_price_regression_prediction"] - sim["adjclose"]) / sim["adjclose"]
        final_sim = sim.merge(fundamentals,on=["year","quarter","ticker"],how="left").dropna()
        final_sim["fundamental_predicted_delta"] = (final_sim["quarterly_fundamental_regression_prediction"] - final_sim["adjclose"]) / final_sim["adjclose"]
        final_sim["bond_predicted_delta"] = (final_sim["weekly_bond_regression_prediction"] - final_sim["adjclose"]) / final_sim["adjclose"]
        strat_db.connect()
        strat_db.drop_table("{}_final_sim".format(dataset))
        strat_db.store_data("{}_final_sim".format("pdr"),final_sim)
        strat_db.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:36<00:00, 36.91s/it]


In [12]:
strat_db.connect()
strat_db.drop_table("{}_final_sim".format(dataset))
strat_db.store_data("{}_final_sim".format("pdr"),sim)
strat_db.close()

In [24]:
final_sim["bond_predicted_delta"].mode()

0   -0.571908
dtype: float64

In [25]:
strat_db.connect()
sim = strat_db.retrieve_data("{}_final_sim".format("pdr"))
strat_db.close()
sim = sim[sim["date"] <= end]
sim.drop("_id",axis=1,inplace=True)
sim["date"].max()

Timestamp('2020-06-26 00:00:00')

In [27]:
sim.sort_values("date")

,date,high,low,open,close,volume,adjclose,ticker,year,week,...,weekly_bond_regression_prediction,weekly_bond_regression_score,weekly_predicted_delta,_id_y,quarterly_fundamental_classification_prediction,quarterly_fundamental_classification_score,quarterly_fundamental_regression_prediction,quarterly_fundamental_regression_score,fundamental_predicted_delta,bond_predicted_delta
0,2017-01-03,46.750000,45.740002,45.930000,46.490002,1739600.0,44.900917,A,2017,1,...,28.138832,0.631366,1599.238057,600dd280da8e71ede7cb6522,1.0,0.75,36.904953,0.073860,-0.178080,-0.373313
71039,2017-01-03,66.970001,65.389999,66.019997,65.610001,3023800.0,63.304447,ROST,2017,1,...,26.500967,0.699123,-1585.557169,600ddd18da8e71ede7cb6677,0.0,1.00,46.142670,0.187644,-0.271099,-0.581373
9839,2017-01-03,34.110001,33.544998,33.785000,33.715000,1652600.0,32.361370,APH,2017,1,...,14.936149,0.743643,6044.861435,600dd3b0da8e71ede7cb6549,1.0,0.75,48.590942,0.076019,0.501511,-0.538457
9840,2017-01-03,34.110001,33.544998,33.785000,33.715000,1652600.0,32.361370,APH,2017,1,...,14.936149,0.743643,-0.200470,600dd3b0da8e71ede7cb6549,1.0,0.75,48.590942,0.076019,0.501511,-0.538457
70684,2017-01-03,185.710007,183.740005,184.820007,184.929993,421400.0,180.349030,ROP,2017,1,...,92.554146,0.773597,-0.056242,600ddd13da8e71ede7cb6676,1.0,0.50,176.772558,0.039588,-0.019831,-0.486805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40640,2020-06-25,1442.900024,1420.000000,1429.900024,1441.329956,1230500.0,1441.329956,GOOG,2020,26,...,962.710388,0.105061,3598.870368,600e6770da8e71ede7cb7bf1,1.0,0.80,1194.429810,0.189842,-0.171300,-0.332068
87897,2020-06-25,176.429993,171.460007,176.300003,176.160004,1103900.0,176.160004,WAT,2020,26,...,152.559860,0.003426,-15457.455458,600e6d47da8e71ede7cb7ce2,1.0,0.60,252.979996,0.133284,0.436081,-0.133970
15875,2020-06-26,1611.709961,1532.829956,1597.300049,1541.250000,601700.0,1541.250000,BKNG,2020,26,...,1132.565796,0.207096,-7802.184104,600e6494da8e71ede7cb7b7f,0.0,0.80,1759.229980,0.107978,0.141431,-0.265164
87898,2020-06-26,176.809998,171.380005,176.160004,174.759995,2124900.0,174.759995,WAT,2020,26,...,152.559860,0.003426,-15581.277727,600e6d47da8e71ede7cb7ce2,1.0,0.60,252.979996,0.133284,0.447585,-0.127032


In [31]:
def backtest(sim,seats,ceiling,weekly,weekly_value,weekly_classification,fundamental,fundamental_value,fundamental_classification,bond,bond_value,bond_classification,delta_req):
        trades = []
        blacklist = pd.DataFrame([{"ticker":"ZZZZZ","start":datetime(2016,4,1),"end":datetime(2016,4,14)}])
        daily_delta_req = float(np.log(1+delta_req)/365)
        if weekly:
            delta_column = "weekly_predicted_delta"
            hpr = 7
        if fundamental and not weekly:
            delta_column = "fundamental_predicted_delta"
            hpr = 90
        if bond and not weekly and not fundamental:
            delta_column = "bond_predicted_delta"
            hpr = 7
        if not weekly_value:
            sim["weekly_predicted_delta"] = sim["weekly_predicted_delta"] * -1
            sim["weekly_price_classification_prediction"] = [not x for x in sim["weekly_price_classification_prediction"]]
        if bond and not bond_value:
            sim["bond_predicted_delta"] = sim["bond_predicted_delta"] * -1
            sim["weekly_bond_classification_prediction"] = [not x for x in sim["weekly_bond_classification_prediction"]]
        if fundamental and not fundamental_value:
            sim["fundamental_predicted_delta"] = sim["fundamental_predicted_delta"] * -1
            sim["quarterly_fundamental_classification_prediction"] = [not x for x in sim["quarterly_fundamental_classification_prediction"]]
        if ceiling:
            for col in sim.columns:
                    if 'delta' in col:
                        sim["{}_abs".format(col)] = [abs(x) for x in sim[col]]
                        filtered_sim = sim[(sim["{}_abs".format(col)] <= 1)]
        else:
            filtered_sim = sim
        filtered_sim = filtered_sim[(filtered_sim[delta_column] >= float((1+daily_delta_req)**hpr) - 1)]
        if fundamental and fundamental_classification:
            filtered_sim = filtered_sim[filtered_sim["quarterly_fundamental_classification_prediction"] == 1]
            filtered_sim = filtered_sim[filtered_sim["quarterly_fundamental_classification_score"] >= 0.7]
        if weekly and weekly_classification:
            filtered_sim = filtered_sim[filtered_sim["weekly_price_classification_prediction"] == 1]
            filtered_sim = filtered_sim[filtered_sim["weekly_price_classification_score"] >= 0.7]
        if bond and bond_classification:
            filtered_sim = filtered_sim[filtered_sim["weekly_bond_classification_prediction"] == 1]
            filtered_sim = filtered_sim[filtered_sim["weekly_bond_classification_score"] >= 0.7]
        if fundamental:
            filtered_sim = filtered_sim[filtered_sim["fundamental_predicted_delta"] >= float((1+daily_delta_req)**90 - 1)]
            filtered_sim = filtered_sim[(filtered_sim["quarterly_fundamental_regression_score"] > 0) & (filtered_sim["quarterly_fundamental_regression_score"] < 0.1)]
        if weekly:
            filtered_sim = filtered_sim[filtered_sim["weekly_predicted_delta"] >= float((1+daily_delta_req)**7 - 1)]
            filtered_sim = filtered_sim[(filtered_sim["weekly_price_regression_score"] > 0) & (filtered_sim["weekly_price_regression_score"] < 0.1)]
        if bond:
            filtered_sim = filtered_sim[filtered_sim["bond_predicted_delta"] >= float((1+daily_delta_req)**7 - 1)]
            filtered_sim = filtered_sim[(filtered_sim["weekly_bond_regression_score"] > 0) & (filtered_sim["weekly_bond_regression_score"] < 0.1)]
        for seat in range(seats):
            date = start
            pv = initial / (seats - 1)
            while date <= end:
                if date >= end:
                    break
                if date.weekday() > 4 \
                        or date == datetime(2020,2,20):
                    date = date + timedelta(days=7-date.weekday())
                blacklist_tickers = blacklist[(blacklist["start"] <= date) & (blacklist["end"] >= date)]["ticker"]
                todays_sim = filtered_sim[ (~filtered_sim["ticker"].isin(blacklist_tickers)) & \
                                          (filtered_sim["date"] == date)]
                if todays_sim.index.size >= 1:
                    offerings = todays_sim.sort_values(delta_column,ascending=False)
                    for offering in range(offerings.index.size):
                        try:
                            trade_ticker = offerings.iloc[offering]["ticker"]
                            try:
                                trade = sim[(sim["ticker"] == trade_ticker) & (sim["date"] > date)].iloc[0]
                                sell_date = trade["date"] + timedelta(days=1)
                                sell_trades = sim[(sim["date"] >= sell_date) & (sim["ticker"] == trade["ticker"])]
                                ## if there aren't any listed prices left for the ticker
                                if sell_trades.index.size < 1:
                                    ## if there aren't any more ticker's left to vet
                                    if offering == offerings.index.size - 1:
                                        date = sell_date + timedelta(days=1)
                                        print(date,"no more stock vets")
                                        break
                                    else:
                                        print(date,"stock had no more listed prices")
                                        continue
                                else:
                                    if date > datetime(2020,10,1):
                                        max_hpr = int((end - date).days) - 1
                                    else:
                                        max_hpr = hpr
                                    sell_trades["gain"] = (sell_trades["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                    sell_trades["hpr"] = [(x - trade["date"]).days for x in sell_trades["date"]]
                                    sell_trades["delta_req"] = [float((1+daily_delta_req) ** (math.floor(x/7)*7) - 1) for x in sell_trades["hpr"]]
                                    hpr_sell_trades = sell_trades[sell_trades["hpr"] <= max_hpr]
                                    hpr_sell_trades["hit"] = hpr_sell_trades["gain"] >= hpr_sell_trades["delta_req"]
                                    delta_hit = hpr_sell_trades[(hpr_sell_trades["hit"] == True)]
                                    ## if we never hit the mark
                                    if delta_hit.index.size < 1:
                                        exits = sell_trades[(sell_trades["adjclose"] >= trade["adjclose"]) & (sell_trades["hpr"]>max_hpr)]
                                        if exits.index.size < 1:
                                            yearly_trades = sell_trades[sell_trades["year"]==date.year]
                                            sell_trade = yearly_trades.iloc[yearly_trades.index.size - 1]
                                        else:
                                            sell_trade = exits.iloc[0]
                                    else:
                                        sell_trade = delta_hit.iloc[0]
                                    trade["sell_price"] = sell_trade["adjclose"]
                                    trade["delta_req"] = sell_trade["delta_req"]
                                    trade["sell_date"] = sell_trade["date"]
                                    trade["sell_delta"] = (sell_trade["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                    trade["hpr"] = sell_trade["hpr"]
                                    trade["seat"] = seat
                                    pv = pv * (1+trade["sell_delta"])
                                    trade["pv"] = pv
                                    blacklist = blacklist.append([{"ticker":trade["ticker"],"start":date,"end":trade["sell_date"]}])
                                    trades.append(trade)
                                    date = trade["sell_date"] + timedelta(days=1)
                                    break
                            except Exception as e:
                                print(date,str(e))
                                date = date + timedelta(days=1)
                                continue
                        except Exception as e:
                            print(date,str(e))
                            date = date + timedelta(days=1)
                            continue
                else:
                    date = date + timedelta(days=1)
                    continue
        return trades

In [38]:
epoch = 0
seats = 21
strat_db.connect()
strat_db.drop_table("accurate_rank_epochs")
iterations = len(datasets) * len(yearly_delta_range) \
                * len(bond_settings) * len(weekly_value_settings) *len(weekly_classification_settings) \
                * len(ceiling_settings) \
                * len(weekly_settings) * len(weekly_value_settings) *len(weekly_classification_settings) \
                * len(fundamental_settings) * len(fundamental_value_settings) *len(fundamental_classification_settings)
print(iterations)
for i in range(iterations):
    strat_db.drop_table("accurate_rank_{}".format(i))
sim["delta"] = [abs(x) for x in sim["weekly_predicted_delta"]]
for ceiling in tqdm(ceiling_settings):
    for weekly in tqdm(weekly_settings):
        for weekly_value in tqdm(weekly_value_settings):
            for weekly_classification in weekly_classification_settings:
                for fundamental in fundamental_settings:
                    for fundamental_classification in fundamental_classification_settings:
                        for fundamental_value in fundamental_value_settings:
                            for bond in bond_settings:
                                for bond_value in bond_value_settings:
                                    for bond_classification in bond_classification_settings:
                                        for delta in yearly_delta_range:
                                            dr = delta/100
                                            epoch_dict = {"epoch":epoch
                                                          ,"dataset":dataset
                                                          ,"dr":dr
                                                          ,"bond_setting":bond
                                                          ,"bond_value":bond_value
                                                          ,"bond_classification":bond_classification
                                                          ,"ceiling":ceiling
                                                          ,"weekly":weekly
                                                          ,"weekly_value":weekly_value
                                                          ,"weekly_classification":weekly_classification
                                                          ,"fundamental":fundamental
                                                          ,"fundamental_value":fundamental_value
                                                          ,"fundamental_classification":fundamental_classification
                                                         }
                                            ts = backtest(sim.copy(),seats,ceiling,weekly,weekly_value,weekly_classification,fundamental,fundamental_value,fundamental_classification,bond,bond_value,bond_classification,dr)
                                            if len(ts) > 0:
                                                strat_db.store_data("accurate_rank_epochs",pd.DataFrame([epoch_dict]))
                                                strat_db.store_data("accurate_rank_{}".format(epoch),pd.DataFrame(ts))
                                            epoch += 1
strat_db.close()

3072


  0%|                                                                                                                                                                                              | 0/2 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                              | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████████████████████████████████████████████████████████████████████████                                                                                          | 1/2 [49:34<49:34, 2974.01s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [1:41:42<00:00, 3051.14s/it]

 50%|████████████████████████████████████████████████████████████████████████████████████████                               

UnboundLocalError: local variable 'delta_column' referenced before assignment